In [2]:
import os

import cv2
import pandas as pd

In [3]:

# Define the input and output directories
input_dir = "image"
output_dir = "image/classified"

# Define the folders to process
folders_to_process = ["gem", "non_gem"]


In [4]:
# Define the output image size
output_size = (224, 224)

In [5]:
def crop_image(image_path):
            img = cv2.imread(image_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (5, 5), 0)
            edges = cv2.Canny(gray, 50, 150)
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            xs = []
            ys = []

            for contour in contours:
                # Calculate the area of the contour
                area = cv2.contourArea(contour)

                # Ignore small contours (noise)
                if area > 0:
                    # Calculate the x, y, w, h coordinates of the bounding rectangle
                    x, y, w, h = cv2.boundingRect(contour)
                    xs.append(x)
                    xs.append(x + w)
                    ys.append(y)
                    ys.append(y + h)

            if len(xs) > 0 and len(ys) > 0:

                w = max(xs) - min(xs)
                h = max(ys) - min(ys)

                l = max(w, h)

                xm = int((max(xs) + min(xs)) / 2)
                ym = int((max(ys) + min(ys)) / 2)

                x1 = xm - int(l/2)
                x2 = xm + int(l/2)
                y1 = ym - int(l/2)
                y2 = ym + int(l/2)

                if x1 < 0:
                    x1 += abs(x1)
                    x2 += abs(x1)
                if y1 < 0:
                    y1 += abs(y1)
                    y2 += abs(y1)

                cropped_img = img[y1:y2, x1:x2]
            else:
                cropped_img = img

            resized_img = cv2.resize(cropped_img, output_size)

            return resized_img

In [6]:
# Iterate through each folder
for folder in folders_to_process:
    # Get the full path to the folder
    folder_path = os.path.join(input_dir, folder)

    # Create the output folder if it doesn't exist
    output_folder_path = os.path.join(output_dir, folder)
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Iterate through each subfolder in the folder
    for subfolder in os.listdir(folder_path):
        # Get the full path to the subfolder
        subfolder_path = os.path.join(folder_path, subfolder)

        # Create the output subfolder if it doesn't exist
        output_subfolder_path = os.path.join(output_folder_path, subfolder)
        if not os.path.exists(output_subfolder_path):
            os.makedirs(output_subfolder_path)

        # Iterate through each image in the subfolder
        if os.path.isdir(subfolder_path):
            for filename in os.listdir(subfolder_path):
                image_path = os.path.join(subfolder_path, filename)
                cv2.imwrite(os.path.join(output_subfolder_path, filename), crop_image(image_path))


In [16]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the dataset path
dataset_path = 'image/classified'

# Define the flowers and non-flowers paths
flowers_path = "image/classified/gem"
non_flowers_path = "image/classified/non_gem"

# Get the list of images in the flowers and non-flowers folders
flowers_images = []
for folder in os.listdir(flowers_path):
    folder_path = os.path.join(flowers_path, folder)
    for image in os.listdir(folder_path):
        flowers_images.append(os.path.join(folder_path, image))

non_flowers_images = []
for folder in os.listdir(non_flowers_path):
    folder_path = os.path.join(non_flowers_path, folder)
    for image in os.listdir(folder_path):
        non_flowers_images.append(os.path.join(folder_path, image))

# Split the images into training, validation, and test sets
train_flowers, temp_flowers = train_test_split(flowers_images, test_size=0.4, random_state=42)
validation_flowers, test_flowers = train_test_split(temp_flowers, test_size=0.5, random_state=42)

train_non_flowers, temp_non_flowers = train_test_split(non_flowers_images, test_size=0.4, random_state=42)
validation_non_flowers, test_non_flowers = train_test_split(temp_non_flowers, test_size=0.5, random_state=42)

# Create the training, validation, and test folders
train_path = os.path.join(dataset_path, 'rcnn/train')
validation_path = os.path.join(dataset_path, 'rcnn/validation')
test_path = os.path.join(dataset_path, 'rcnn/test')

if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(validation_path):
    os.makedirs(validation_path)
if not os.path.exists(test_path):
    os.makedirs(test_path)

# Create the flowers and non-flowers folders inside the training, validation, and test folders
train_flowers_path = os.path.join(train_path, 'gem')
train_non_flowers_path = os.path.join(train_path, 'non_gem')
validation_flowers_path = os.path.join(validation_path, 'gem')
validation_non_flowers_path = os.path.join(validation_path, 'non_gem')
test_flowers_path = os.path.join(test_path, 'gem')
test_non_flowers_path = os.path.join(test_path, 'non_gem')

if not os.path.exists(train_flowers_path):
    os.makedirs(train_flowers_path)
if not os.path.exists(train_non_flowers_path):
    os.makedirs(train_non_flowers_path)
if not os.path.exists(validation_flowers_path):
    os.makedirs(validation_flowers_path)
if not os.path.exists(validation_non_flowers_path):
    os.makedirs(validation_non_flowers_path)
if not os.path.exists(test_flowers_path):
    os.makedirs(test_flowers_path)
if not os.path.exists(test_non_flowers_path):
    os.makedirs(test_non_flowers_path)

# Move the images to the training, validation, and test folders
for image in train_flowers:
    filename = os.path.basename(image)
    shutil.copy(image, train_flowers_path)

for image in train_non_flowers:
    filename = os.path.basename(image)
    shutil.copy(image, train_non_flowers_path)

for image in validation_flowers:
    filename = os.path.basename(image)
    shutil.copy(image, validation_flowers_path)

for image in validation_non_flowers:
    filename = os.path.basename(image)
    shutil.copy(image, validation_non_flowers_path)

for image in test_flowers:
    filename = os.path.basename(image)
    shutil.copy(image, test_flowers_path)

for image in test_non_flowers:
    filename = os.path.basename(image)
    shutil.copy(image, test_non_flowers_path)

In [17]:

# Define the image dimensions
img_height, img_width = 224, 224

# Define the batch size
batch_size = 32

# Create data generators for training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_path, 'rcnn/train'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes=['non_gem', 'gem']
)

validation_generator = validation_datagen.flow_from_directory(
    os.path.join(dataset_path, 'rcnn/validation'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes=['non_gem', 'gem']
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(dataset_path, 'rcnn/test'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    classes=['non_gem', 'gem']
)

Found 526 images belonging to 2 classes.
Found 176 images belonging to 2 classes.
Found 176 images belonging to 2 classes.


In [18]:

# Create the RCNN model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

# Freeze the base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10)

test_loss, test_acc = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size)

print(f'Test loss: {test_loss:.3f}')
print(f'Test accuracy: {test_acc:.3f}')

# Save the model
model.save('rcnn_model.h5')

Epoch 1/10


/home/hmr/SDK/Python/_venv/Gem/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 196s 12s/step - accuracy: 0.8076 - loss: 1.2456 - val_accuracy: 0.9375 - val_loss: 0.2431
Epoch 2/10
 1/16 ━━━━━━━━━━━━━━━━━━━━ 2:07 8s/step - accuracy: 0.9688 - loss: 0.1502

/home/hmr/SDK/Python/_venv/Gem/.venv/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


16/16 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - accuracy: 0.9688 - loss: 0.1502 - val_accuracy: 0.9375 - val_loss: 0.3202
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 201s 13s/step - accuracy: 0.9365 - loss: 0.2412 - val_accuracy: 0.9312 - val_loss: 0.3036
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9688 - loss: 0.1123 - val_accuracy: 0.9312 - val_loss: 0.3267
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 259s 14s/step - accuracy: 0.9793 - loss: 0.0536 - val_accuracy: 0.9688 - val_loss: 0.0769
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.9688 - loss: 0.0990 - val_accuracy: 0.9688 - val_loss: 0.0785
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 203s 13s/step - accuracy: 0.9966 - loss: 0.0099 - val_accuracy: 0.9625 - val_loss: 0.1012
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 81s 5s/step - accuracy: 1.0000 - loss: 0.0251 - val_accuracy: 0.9625 - val_loss: 0.0883
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 260s 14s/step - accuracy: 0.9975 - loss: 0.0130 - val_accuracy: 0.9625 - val_loss: 0.

Test loss: 0.086
Test accuracy: 0.975


In [8]:
# Load the saved model
from tensorflow.keras.models import load_model
model = load_model('rcnn_model.h5')

# Load the image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the image from a file path
def load_image(path):
    img = crop_image(path)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# Make a prediction on the image
def make_prediction(path):
    img_array = load_image(path)
    prediction = model.predict(img_array)
    return prediction

print("======= Predictions ======")
paths = os.listdir('image/test')
predictions = []
for name in paths:
    image_path = os.path.join('image/test', name)
    prediction = list(make_prediction(image_path))
    score = float(prediction[0][0])
    is_gem = score > 0.5
    predictions.append([image_path, f"{'Gem' if is_gem else 'Non-gem'}", score])

    #
    # print(f"{image_path.rjust(12)} -> {prediction} -> {is_gem}")
    # Convert the prediction to a class label
    # if prediction[0][0] > 0.5:
    #     print("The image is a gem.")
    # else:
    #     print("The image is not a gem.")


======= Predictions ======
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 806ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 754ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 781ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 594ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 618ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 607ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step


In [9]:
df = pd.DataFrame(predictions, columns=['Image', 'Prediction', 'Score'])
df

,Image,Prediction,Score
0,image/test/IMG_20250121_122222.jpg,Gem,0.908380
1,image/test/ruby.png,Gem,0.999978
2,image/test/coin.png,Non-gem,0.315042
3,image/test/pen.png,Gem,0.990088
4,image/test/gem.png,Gem,0.999999
5,image/test/sapphire.png,Gem,0.999988
6,image/test/coin-2.JPG,Non-gem,0.070320
7,image/test/tree.png,Gem,0.993571
8,image/test/car.png,Gem,0.999853
9,image/test/IMG_9930.JPG,Non-gem,0.004736
